In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-08 21:57:19.006141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 21:57:19.006203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 21:57:19.009784: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 21:57:19.029449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 21:57:21.673824: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../melanoma_cancer_dataset"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/9605
[INFO]: Processed 2000/9605
[INFO]: Processed 3000/9605
[INFO]: Processed 4000/9605
[INFO]: Processed 5000/9605
[INFO]: Processed 6000/9605
[INFO]: Processed 7000/9605
[INFO]: Processed 8000/9605
[INFO]: Processed 9000/9605
(9605, 224, 224, 3)
(9605,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-08 21:58:29.610772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


301/301 [==============================] - 581s 2s/step
(9605, 100352)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9354838709677419
Sensitivity: 0.9327548806941431
Specificity: 0.938
AUC-ROC: 0.9353774403470715
MCC: 0.8707548806941432
Precision: 0.9327548806941431
F1 Score: 0.9327548806941431

[INFO] Fold 2 / 10 for LR
Accuracy: 0.9375650364203955
Sensitivity: 0.9262472885032538
Specificity: 0.948
AUC-ROC: 0.9371236442516269
MCC: 0.8749612140765305
Precision: 0.9426048565121413
F1 Score: 0.9343544857768051

[INFO] Fold 3 / 10 for LR
Accuracy: 0.9354838709677419
Sensitivity: 0.9197396963123644
Specificity: 0.95
AUC-ROC: 0.9348698481561822
MCC: 0.8708966225330048
Precision: 0.9443207126948775
F1 Score: 0.9318681318681319

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9167533818938606
Sensitivity: 0.89587852494577
Specificity: 0.936
AUC-ROC: 0.915939262472885
MCC: 0.8334710540356263
Precision: 0.9280898876404494
F1 Score: 0.9116997792494481

[INFO] Fold 5 / 10 for LR
Accuracy: 0.9167533818938606
Sensitivity: 0.8872017353579176
Specificity: 0.944
AUC-ROC: 0.9156008676789588
MCC: 0.833941418727808
P

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8314255983350677
Sensitivity: 0.754880694143167
Specificity: 0.902
AUC-ROC: 0.8284403470715834
MCC: 0.6664800553695792
Precision: 0.8765743073047859
F1 Score: 0.8111888111888113

[INFO] Fold 2 / 10 for NB
Accuracy: 0.8480749219562955
Sensitivity: 0.8112798264642083
Specificity: 0.882
AUC-ROC: 0.8466399132321041
MCC: 0.6961184081628988
Precision: 0.8637413394919169
F1 Score: 0.83668903803132

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8428720083246618
Sensitivity: 0.7744034707158352
Specificity: 0.906
AUC-ROC: 0.8402017353579176
MCC: 0.6886264581472906
Precision: 0.8836633663366337
F1 Score: 0.8254335260115606

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8345473465140478
Sensitivity: 0.7982646420824295
Specificity: 0.868
AUC-ROC: 0.8331323210412147
MCC: 0.6688551245775669
Precision: 0.847926267281106
F1 Score: 0.8223463687150838

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8366285119667014
Sensitivity: 0.7570498915401301
Specificity: 0.91
AUC-ROC: 0.833524945770065

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.9053069719042663
Sensitivity: 0.8503253796095445
Specificity: 0.956
AUC-ROC: 0.9031626898047722
MCC: 0.8134895171078206
Precision: 0.9468599033816425
F1 Score: 0.896

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.9125910509885536
Sensitivity: 0.8633405639913232
Specificity: 0.958
AUC-ROC: 0.9106702819956616
MCC: 0.8274696662965947
Precision: 0.9498806682577565
F1 Score: 0.9045454545454545

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.9146722164412071
Sensitivity: 0.8481561822125814
Specificity: 0.976
AUC-ROC: 0.9120780911062907
MCC: 0.8344020512311798
Precision: 0.9702233250620348
F1 Score: 0.9050925925925926

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.899063475546306
Sensitivity: 0.841648590021692
Specificity: 0.952
AUC-ROC: 0.896824295010846
MCC: 0.8011778494740814
Precision: 0.941747572815534
F1 Score: 0.8888888888888888

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.8938605619146722
Sensitivity: 0.8177874186550976
Specificity: 0.964
AUC-ROC: 0.8908937093275489
MCC: 

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9198751300728408
Sensitivity: 0.9088937093275488
Specificity: 0.93
AUC-ROC: 0.9194468546637743
MCC: 0.839480275746178
Precision: 0.9229074889867841
F1 Score: 0.9158469945355191

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9209157127991675
Sensitivity: 0.9067245119305857
Specificity: 0.934
AUC-ROC: 0.9203622559652928
MCC: 0.8416195509353102
Precision: 0.926829268292683
F1 Score: 0.9166666666666666

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.918834547346514
Sensitivity: 0.8915401301518439
Specificity: 0.944
AUC-ROC: 0.917770065075922
MCC: 0.8379831127365405
Precision: 0.9362186788154897
F1 Score: 0.9133333333333333

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9167533818938606
Sensitivity: 0.89587852494577
Specificity: 0.936
AUC-ROC: 0.915939262472885
MCC: 0.8334710540356263
Precision: 0.9280898876404494
F1 Score: 0.9116997792494481

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.9021852237252862
Sensitivity: 0.8720173535

In [ ]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 30.098457 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18559740
[LightGBM] [Info] Number of data points in the train set: 8644, number of used features: 100048
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.479408 -> initscore=-0.082416
[LightGBM] [Info] Start training from score -0.082416
Accuracy: 0.9354838709677419
Sensitivity: 0.9088937093275488
Specificity: 0.96
AUC-ROC: 0.9344468546637742
MCC: 0.8714342123186546
Precision: 0.9544419134396356
F1 Score: 0.9311111111111111

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 28.867862 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] To

In [ ]:
# Save results to a CSV file
results_df.to_csv('DS4_ResNet50.csv', index=False)

In [ ]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.927641,0.915965,0.9384,0.927182,0.855148,0.932005,0.923856,2319.765625,3258.295190
1,NB,0.837897,0.775457,0.8954,0.835429,0.677979,0.872560,0.820924,3677.289062,202.497991
2,KNN,0.905674,0.846908,0.9598,0.903354,0.814872,0.951114,0.895909,3688.039062,292.647123
3,Random Forest,0.917648,0.897505,0.9362,0.916853,0.835285,0.928391,0.912648,3721.867188,263.661813
4,LGBM,0.928371,0.911186,0.9442,0.927693,0.856741,0.937766,0.924234,4886.246094,10830.620726
